# Runtime for Various Batchsizes

## Imports and Definitions

In [ ]:
import os
import time
import torch
import cvxpy as cp
import pandas as pd

from SCMP import SCMP
import DataGeneration as data
from CommonFunctions import *
import Presentation

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)

PATH = "./Results/runtime_varying_batchsizes"
if not os.path.exists(PATH):
    os.makedirs(PATH)

## Data Generation

In [ ]:
X_dim = 5
N_samples = 1024
validation_percent = 0.25

X, Y = data.gen_sklearn_data(X_dim, N_samples)
X, Y, Xval, Yval = data.split_data(X, Y, validation_percent)

## Training

In [ ]:
batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
epochs = 5

In [ ]:
results = {
    "batch_sizes": [],
    "fit_times": [],
    "ccp_times": []
}

for batch_size in batch_sizes:
    print(f"Batch size: {batch_size}.")
    scmp = SCMP(X_dim, batch_size, cost_fn=quad_cost_cvxpy_batched, cost_fn_not_batched=quad_cost_cvxpy_not_batched, strategic=True)
    scmp.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 1e-1},
             epochs=epochs, verbose="batches", path=PATH, model_name=f"batch_size_{batch_size}")
    
    results["batch_sizes"].append(batch_size)
    results["fit_times"].append(scmp.fit_time)
    results["ccp_times"].append(scmp.ccp_time)
    pd.DataFrame(results).to_csv(f"{PATH}/timing_results_without_wrap.csv")

In [ ]:
batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256]

results = {
    "batch_sizes": [],
    "fit_times": [],
    "ccp_times": []
}

for batch_size in batch_sizes:
    print(f"Batch size: {batch_size}.")
    # Note that we don't give the model the unbatched quad_cost, SCMP wraps the batched version to create it.
    scmp = SCMP(X_dim, batch_size, cost_fn=quad_cost_cvxpy_batched, strategic=True)
    scmp.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 1e-1},
             epochs=epochs, verbose="batches", path=PATH, model_name=f"batch_size_{batch_size}_with_wrap")
    
    results["batch_sizes"].append(batch_size)
    results["fit_times"].append(scmp.fit_time)
    results["ccp_times"].append(scmp.ccp_time)
    pd.DataFrame(results).to_csv(f"{PATH}/timing_results_with_wrap.csv")

## Show Results

In [ ]:
Presentation.show_runtime_results()